In [1]:
!pip install rebound reboundx astroquery astropy numpy pandas
import rebound
import reboundx
import numpy as np
import os
import pandas as pd
from astroquery.jplhorizons import Horizons
from astropy.time import Time
import warnings
from astropy.utils.exceptions import ErfaWarning
warnings.simplefilter('ignore', ErfaWarning)

/home/vikoprtm/jupyter_env/lib/python3.12/site-packages/reboundx/__init__.py:29: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
from erfa import ErfaWarning

 [astropy.utils.exceptions]


In [2]:
# Inisiasi waktu awal simulasi dan parameter waktu
initdate = "2011-01-01 00:00:00"

# --- PARAMETERS ---
JD_start = 2456400.5     # Waktu awal epoch dalam Julian Date 
JD_ref = 2451545.0       # Epoch referensi dalam Julian Date (J200) 
days_per_year = 365.25

# Konversi JD_start ke waktu relatif J2000
t0 = (JD_start - JD_ref) / days_per_year

In [3]:
# ---[2] Setup simulasi REBOUND ---
sim = rebound.Simulation()
sim.units = ('AU', 'yr', 'Msun')
sim.integrator = "ias15"      # integrator akurat untuk jangka panjang

In [4]:
# ---[3] Tambahkan Matahari dengan massa tambahan planet dalam ---
M_sun = 1.0
M_inner = (1.66e-7 + 2.45e-6 + 3.00e-6 + 3.23e-7)  # Merkurius–Mars
M_sun_new = M_sun + M_inner
sim.add(m=M_sun_new)

In [5]:
# ---[4] Tambahkan planet raksasa saja ---
for planet in ["Jupiter", "Saturn", "Uranus", "Neptune"]:
    sim.add(planet, date=initdate)

Searching NASA Horizons for 'Jupiter'... 
Found: Jupiter Barycenter (5) (chosen from query 'Jupiter')
Searching NASA Horizons for 'Saturn'... 
Found: Saturn Barycenter (6) (chosen from query 'Saturn')
Searching NASA Horizons for 'Uranus'... 
Found: Uranus Barycenter (7) (chosen from query 'Uranus')
Searching NASA Horizons for 'Neptune'... 
Found: Neptune Barycenter (8) (chosen from query 'Neptune')


In [6]:
# ---[5] Data asteroid keluarga Koronis ---
asteroids = [
    {"name": "Kulin",  "a": 2.863, "e": 0.051, "inc": np.radians(3.222),
     "Omega": np.radians(104.10), "omega": np.radians(322.18), "M": np.radians(265.38),
     "albedo": 0.268, "D": 11.744},
    {"name": "Waland", "a": 2.747, "e": 0.054, "inc": np.radians(3.484),
     "Omega": np.radians(180.40), "omega": np.radians(294.72), "M": np.radians(317.02),
     "albedo": 0.197, "D": 9.039},
    {"name": "Yorii",  "a": 3.070, "e": 0.2455, "inc": np.radians(5.505),
     "Omega": np.radians(111.90), "omega": np.radians(14.811), "M": np.radians(357.21),
     "albedo": 0.091, "D": 12.438}
]

for ast in asteroids:
    sim.add(a=ast["a"], e=ast["e"], inc=ast["inc"],
            Omega=ast["Omega"], omega=ast["omega"], M=ast["M"])
sim.move_to_com()

In [7]:
# ---[6] Parameter fisik (Tabel 3.2) ---
density = 2.4e6
Gamma = 8.72e-10
rotation_period = 6 / 24.0 / 365.25
emissivity = 0.9
q = 0.39
k = 0.25

In [8]:
# ---[7] Efek Yarkovsky dari REBOUNDx ---
rebx = reboundx.Extras(sim)
yark = rebx.load_force("yarkovsky_effect")

In [9]:
# konstanta fisis
c = 63241.077
stef_boltz = 8.96e-16
lstar = 2.7e-4
yark.params["ye_c"] = c
yark.params["ye_stef_boltz"] = stef_boltz
yark.params["ye_lstar"] = lstar
rebx.add_force(yark)

In [10]:
# ---[8] Set parameter Yarkovsky per asteroid ---
ps = sim.particles
offset = len(ps) - len(asteroids)

print("\n=== DATA FISIS ASTEROID ===")
for i, ast in enumerate(asteroids):
    idx = offset + i
    pv = ast["albedo"]
    AB = q * pv
    radius = (ast["D"]/2) * 6.6846e-9

    print(f"{ast['name']}: pv={pv:.3f}, AB={AB:.4f}, radius={radius:.2e} AU")

    ps[idx].r = radius
    ps[idx].params["ye_flag"] = 0
    ps[idx].params["ye_body_density"] = density
    ps[idx].params["ye_albedo"] = AB
    ps[idx].params["ye_emissivity"] = emissivity
    ps[idx].params["ye_k"] = k
    ps[idx].params["ye_thermal_inertia"] = Gamma
    ps[idx].params["ye_rotation_period"] = rotation_period
    ps[idx].params["ye_spin_axis_x"] = 0
    ps[idx].params["ye_spin_axis_y"] = 0
    ps[idx].params["ye_spin_axis_z"] = -1


=== DATA FISIS ASTEROID ===
Kulin: pv=0.268, AB=0.1045, radius=3.93e-08 AU
Waland: pv=0.197, AB=0.0768, radius=3.02e-08 AU
Yorii: pv=0.091, AB=0.0355, radius=4.16e-08 AU


In [11]:
# =====================================
# === INTEGRASI (10 juta tahun) ===
# =====================================

# output: setiap 1000 tahun selama 10 juta tahun → 10.000 data
n_outputs = 10000000
years_per_step = 1
total_time = n_outputs * years_per_step

# waktu awal
current_time = t0

# File CSV tujuan
output_filename = "output4_retro_yarkovsky_10Myr.csv"

# Hapus file lama bila ada
if os.path.exists(output_filename):
    os.remove(output_filename)

print("\n=== MULAI INTEGRASI 10 JUTA TAHUN ===\n")

# ====== FUNGSI: state vector → orbital elements ======
def kepler_from_state(px, py, pz, vx, vy, vz, mu):
    r_vec = np.array([px, py, pz], float)
    v_vec = np.array([vx, vy, vz], float)

    r = np.linalg.norm(r_vec)
    v = np.linalg.norm(v_vec)

    h_vec = np.cross(r_vec, v_vec)
    h = np.linalg.norm(h_vec)

    # inclination
    inc = np.arccos(h_vec[2] / h)

    # node vector
    k = np.array([0, 0, 1])
    n_vec = np.cross(k, h_vec)
    n = np.linalg.norm(n_vec)

    # Ω
    if n != 0:
        Omega = np.arccos(np.clip(n_vec[0] / n, -1, 1))
        if n_vec[1] < 0:
            Omega = 2*np.pi - Omega
    else:
        Omega = 0.0

    # e
    rv = np.dot(r_vec, v_vec)
    e_vec = (1/mu)*((v*v - mu/r)*r_vec - rv*v_vec)
    e = np.linalg.norm(e_vec)

    # ω
    if n != 0 and e != 0:
        cosw = np.dot(n_vec, e_vec) / (n * e)
        cosw = np.clip(cosw, -1, 1)
        omega = np.arccos(cosw)
        if e_vec[2] < 0:
            omega = 2*np.pi - omega
    else:
        omega = 0

    # true anomaly ν
    if e != 0:
        cosv = np.dot(e_vec, r_vec) / (e * r)
        cosv = np.clip(cosv, -1, 1)
        nu = np.arccos(cosv)
        if rv < 0:
            nu = 2*np.pi - nu
    else:
        nu = 0

    # semi-major axis a
    a = 1 / (2/r - v*v/mu)

    # mean anomaly
    if e < 1:
        E = 2*np.arctan(np.tan(nu/2)/np.sqrt((1+e)/(1-e)))
        M = E - e*np.sin(E)
        M = np.mod(M, 2*np.pi)
    else:
        M = nu

    return a, e, inc, Omega, omega, M

# ======================================
# LOOP UTAMA INTEGRASI
# ======================================

particles = sim.particles
offset = len(particles) - len(asteroids)
G = 4*np.pi*np.pi   # AU^3 / (yr^2 * Msun)

for j in range(n_outputs):
    # integrasi
    sim.integrate(current_time)

    # data baris CSV
    row = {
        "step": j,
        "time_years": current_time
    }

    # hitung orbit untuk tiap asteroid
    for i, ast in enumerate(asteroids):
        idx = offset + i
        p = particles[idx]

        px, py, pz = p.x, p.y, p.z
        vx, vy, vz = p.vx, p.vy, p.vz

        mu = G*(particles[0].m + p.m)

        a, e, inc, Omega, omega, M = kepler_from_state(px, py, pz, vx, vy, vz, mu)

        row[f"{ast['name']}_a"]   = a
        row[f"{ast['name']}_e"]   = e
        row[f"{ast['name']}_inc"] = np.degrees(inc)

    # simpan ke CSV
    pd.DataFrame([row]).to_csv(
        output_filename,
        mode='a',
        index=False,
        header=(j == 0)
    )

    # next time
    current_time += years_per_step

    if j % 200 == 0:
        print(f"Step {j}/{n_outputs}   Time = {current_time/1e6:.3f} Myr")

print("\n=== SELESAI ===")
print(f"Output disimpan di: {output_filename}")



=== MULAI INTEGRASI 10 JUTA TAHUN ===

Step 0/10000000   Time = 0.000 Myr
Step 200/10000000   Time = 0.000 Myr
Step 400/10000000   Time = 0.000 Myr
Step 600/10000000   Time = 0.001 Myr
Step 800/10000000   Time = 0.001 Myr
Step 1000/10000000   Time = 0.001 Myr
Step 1200/10000000   Time = 0.001 Myr
Step 1400/10000000   Time = 0.001 Myr
Step 1600/10000000   Time = 0.002 Myr
Step 1800/10000000   Time = 0.002 Myr
Step 2000/10000000   Time = 0.002 Myr
Step 2200/10000000   Time = 0.002 Myr
Step 2400/10000000   Time = 0.002 Myr
Step 2600/10000000   Time = 0.003 Myr
Step 2800/10000000   Time = 0.003 Myr
Step 3000/10000000   Time = 0.003 Myr
Step 3200/10000000   Time = 0.003 Myr
Step 3400/10000000   Time = 0.003 Myr
Step 3600/10000000   Time = 0.004 Myr
Step 3800/10000000   Time = 0.004 Myr
Step 4000/10000000   Time = 0.004 Myr
Step 4200/10000000   Time = 0.004 Myr
Step 4400/10000000   Time = 0.004 Myr
Step 4600/10000000   Time = 0.005 Myr
Step 4800/10000000   Time = 0.005 Myr
Step 5000/1000000